In [1]:
# import dependencies
import datetime as dt
import pandas as pd
import numpy as np
import requests, io, zipfile, os, time
import dask.dataframe as dd
# import urllib.request
# from urllib.error import URLError


In [2]:
# format of url = "https://s3.amazonaws.com/tripdata/201306-citibike-tripdata.zip"
url_base = 'https://s3.amazonaws.com/tripdata/'

root = 'Resources'

result_files = []
rename_files = []

In [3]:
# year_list = [2022, 2021]
month_list = [1,2,3,4,5,6,7,8,9,10,11,12]
year_list = [2022]
# month_list = [1,2,3]
    
# of months to loop through
for y in year_list:
    url_year = y
    
    for m in month_list:
        url_month = m
        url_file_prefix = f'{url_year:04d}{url_month:02d}'

        # set up file name
        new_file = url_file_prefix + '.csv'

        # directory files same through 2016, changed for 2017
        url_file_2016 = '-citibike-tripdata.zip'
        url_file_2017 = '-citibike-tripdata.csv.zip'

        # build the proper url to get data from files in the directory
        if url_year >= 2017:
            url = url_base + url_file_prefix + url_file_2017
        else:
            url = url_base + url_file_prefix + url_file_2016
        print(url) 

        # make request for csv file
        r = requests.get(url)
        un_zipped = zipfile.ZipFile(io.BytesIO(r.content))

        # download only the existing file in the root directory
        with un_zipped as zf:
            for file in zf.namelist():

                # not in root directory, pass, otherwise save
                if file.startswith('_'):
                    pass
                else:
                    zf.extract(file, root)                        
                    result_files.append(file)
                    rename_files.append(new_file)


https://s3.amazonaws.com/tripdata/202201-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/202202-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/202203-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/202204-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/202205-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/202206-citibike-tripdata.csv.zip


BadZipFile: File is not a zip file

In [ ]:
# Function to rename files for clarity and easier looping to combine the data
##  https://www.tutorialspoint.com/rename-multiple-files-using-python

def main():
    for i, file in enumerate(os.listdir(root)):
        source = f'{root}/{result_files[i]}'
        destination = f'{root}/{rename_files[i]}'
        os.rename(source, destination)
# Driver Code
if __name__ == '__main__':     
    # Calling main() function
    main()

In [ ]:
# create list for csv reads
path_list = []
for i, csv_file in enumerate(rename_files):
    path = f'{root}/{csv_file}'
    path_list.append(path)

# for curiosity, time taken to read data
## dask documentation from https://www.geeksforgeeks.org/working-with-large-csv-files-in-python/
s_time = time.time()

# Read CSV files from List
df = pd.concat(map(pd.read_csv, path_list))

e_time = time.time()
print("Processed in ", ((e_time-s_time)/60), "minutes")

In [ ]:
df

In [ ]:
df.shape